In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('max_colwidth',1)

In [2]:
deliveiry =pd.read_csv(r"C:\Users\ANAIKUTTY\Desktop\Kutty_Spr\Masterclass\MAS-ACA-Masterclass-main\CSV Files\deliveries.csv")
matches = pd.read_csv(r"C:\Users\ANAIKUTTY\Desktop\Kutty_Spr\Masterclass\MAS-ACA-Masterclass-main\CSV Files\matches.csv")

In [3]:
df = deliveiry.copy()
mdf = matches.copy()

In [4]:
mdf.rename(columns={'id':'match_id'},inplace=True)
df = df.merge(mdf, on='match_id' , how= 'left')

In [5]:
df['phase'] = df['over'].apply(lambda x:'Powerplay' if x <= 6 else ( 'Middle' if x <= 15 else 'Death Over'))

In [6]:
def custom(df,current_opp,current_venue,current_phase):

    #Applying Filters To The Dataset
    df = df[df['bowling_team'] ==current_opp]
    df = df[df['venue'] == current_venue]
    df = df[df['phase'] == current_phase]
    

    df['isdot'] = df['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
    df['isone'] = df['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
    df['istwo'] = df['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
    df['isthree'] = df['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
    df['isfour'] = df['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
    df['issix'] = df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)

    #Phases

    df['phase'] = df['over'].apply(lambda x:'Powerplay' if x <= 6 else ( 'Middle' if x <= 15 else 'Death Over'))

    fil_df = pd.DataFrame(df.groupby('batsman').agg({'match_id':'nunique','batsman_runs':'sum','ball':'count','player_dismissed':'count','isfour':'sum','issix':'sum','isdot':'sum'})).reset_index()

    fil_df = fil_df.rename(columns= {'match_id':'innings','batsman_runs':'runs','player_dismissed':'outs','isfour':'four','issix':'six','isdot':'dots'})

    fil_df = fil_df[fil_df['innings']>1]

    fil_df['RPI'] = fil_df['runs'] / fil_df['innings']
    fil_df['strikerate'] = 100*fil_df['runs'] / fil_df['ball']
    fil_df['BPD'] = np.where(fil_df['outs'] == 0,fil_df['ball'],fil_df['ball']/fil_df['outs'])
    fil_df['DOT_percentage'] = fil_df['dots'] / fil_df['ball']

    #MCDM
    
    wt_sr,wt_rpi,wt_bpd,wt_dot = .13,.27,.16,.45

    #x*x

    fil_df['calc_sr'] = fil_df['strikerate'] * fil_df['strikerate']
    fil_df['calc_rpi'] = fil_df['RPI'] * fil_df['RPI']
    fil_df['calc_bpd'] = fil_df['BPD'] * fil_df['BPD']
    fil_df['calc_dot'] = fil_df['DOT_percentage'] * fil_df['DOT_percentage']

    #sqrt((X1**2)+(X2**2)+(X3**2)....(Xn**2))

    sq_sr = np.sqrt(sum([fil_df['calc_sr']]))
    sq_rpi = np.sqrt(sum([fil_df['calc_rpi']]))
    sq_bpd = np.sqrt(sum([fil_df['calc_bpd']]))
    sq_dot = np.sqrt(sum([fil_df['calc_dot']]))

    #(x*x) / sqrt((X1**2)+(X2**2)+(X3**2)....(Xn**2))

    fil_df['calc_sr'] = fil_df['calc_sr']   / sq_sr
    fil_df['calc_rpi'] = fil_df['calc_rpi'] / sq_rpi
    fil_df['calc_bpd'] = fil_df['calc_bpd'] / sq_bpd
    fil_df['calc_dot'] = fil_df['calc_dot'] / sq_dot

    #(x*x) / sqrt((X1**2)+(X2**2)+(X3**2)....(Xn**2)) * wt_criteria

    fil_df['calc_sr'] = fil_df['calc_sr']   * wt_sr
    fil_df['calc_rpi'] = fil_df['calc_rpi'] * wt_rpi
    fil_df['calc_bpd'] = fil_df['calc_bpd'] * wt_bpd
    fil_df['calc_dot'] = fil_df['calc_dot'] * wt_dot

    #best value , worst value

    best_sr , worst_sr = max(fil_df['calc_sr']) , min(fil_df['calc_sr'])
    best_rpi , worst_rpi = max(fil_df['calc_rpi']) , min(fil_df['calc_rpi'])
    best_bpd , worst_bpd = max(fil_df['calc_bpd']) , min(fil_df['calc_bpd'])
    best_dot , worst_dot = min(fil_df['calc_dot']) , max(fil_df['calc_dot'])


    #DEV Best SQRT

    fil_df['dev_best_sr'] = (fil_df['calc_sr']-best_sr)**2
    fil_df['dev_best_rpi'] = (fil_df['calc_rpi']-best_sr)**2
    fil_df['dev_best_bpd'] = (fil_df['calc_bpd']-best_sr)**2
    fil_df['dev_best_dot'] = (fil_df['calc_dot']-best_sr)**2

    fil_df['dev_best_sqrt'] = fil_df['dev_best_sr'] + fil_df['dev_best_rpi'] + fil_df['dev_best_bpd'] + fil_df['dev_best_dot']

    #DEV Worst sqrt

    fil_df['dev_worst_sr'] = (fil_df['calc_sr']-worst_sr)**2
    fil_df['dev_worst_rpi'] = (fil_df['calc_rpi']-worst_sr)**2
    fil_df['dev_worst_bpd'] = (fil_df['calc_bpd']-worst_sr)**2
    fil_df['dev_worst_dot'] = (fil_df['calc_dot']-worst_sr)**2

    fil_df['dev_worst_sqrt'] = fil_df['dev_worst_sr'] + fil_df['dev_worst_rpi'] + fil_df['dev_worst_bpd'] + fil_df['dev_worst_dot']

    fil_df['score'] = fil_df['dev_worst_sqrt'] / (fil_df['dev_worst_sqrt'] + fil_df['dev_best_sqrt'])

    return fil_df

In [7]:
cus_df = custom(df,'Chennai Super Kings','MA Chidambaram Stadium, Chepauk','Middle')

In [8]:
cus_df.sort_values(by='score',ascending=False)[['batsman','innings','runs','RPI','strikerate','DOT_percentage','score']]

,batsman,innings,runs,RPI,strikerate,DOT_percentage,score
98,SE Marsh,3,104,34.666667,192.592593,0.185185,0.313606
103,SR Watson,4,121,30.250000,172.857143,0.228571,0.229739
4,AB de Villiers,4,88,22.000000,144.262295,0.278689,0.180733
76,NV Ojha,3,64,21.333333,156.097561,0.292683,0.163040
36,G Gambhir,3,86,28.666667,117.808219,0.287671,0.158145
69,MS Bisla,3,89,29.666667,134.848485,0.303030,0.140692
113,Y Venugopal Rao,3,67,22.333333,131.372549,0.313725,0.138645
7,AD Russell,2,37,18.500000,137.037037,0.259259,0.123478
27,DJ Bravo,2,26,13.000000,130.000000,0.350000,0.100601
107,V Kohli,7,175,25.000000,116.666667,0.333333,0.099549
